In [88]:
# %matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
gemd_tools_path = "/srv/hemi01-j01/gemd_tools"
sys.path.append(gemd_tools_path)
from workflow.Folder import Folder
from workflow.TAMUWorkflow import TAMUWorkflow
from workflow.Workflow import Workflow
from utilities.tools import  plot_graph
import os
from pathlib import Path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
iteration_count = 2
iteration = "AAB"
source_folder = "../data/"
output_folder = "data/{}".format(iteration)
sample_data_folder = "../Sample Data/Iteration{}_{}".format(iteration_count, iteration)
path = os.path.join(source_folder, iteration)
w_aab = TAMUWorkflow(path, output_folder=output_folder, iteration=iteration, sample_data_folder=sample_data_folder)
w_aab.testing_mode=True
# iteration_count = 3
# iteration = "AAC"
# source_folder = "../data/"
# output_folder = "data/{}".format(iteration)
# sample_data_folder = "../Sample Data/Iteration{}_{}".format(iteration_count, iteration)
# path = os.path.join(source_folder, iteration)
# w_aac = TAMUWorkflow(path, output_folder=output_folder, iteration=iteration, sample_data_folder=sample_data_folder)

In [90]:
w_aab.build_model()
# w_aac.build_model()
# w_aac.link_prior(w_aab)

(('start_date', '2022-11-03'), ('finish_date', '2022-11-05'), ('time_spent', '3'), ('Full Ingot::7', '175'), ('Full Ingot::6', '145'), ('Full Ingot::5', '145'), ('Full Ingot::4', '145'), ('Full Ingot::3', '145'), ('Full Ingot::2', '145'), ('Full Ingot::1', '110'), ('3 Part Sections::3', '110'), ('3 Part Sections::2', '90'), ('3 Part Sections::1', '90'))
{
   "cartaMetadata": {
      "schemaUsed": "vam-synthesis-details-v1.json",
      "updated": "2023-01-19 17:43:38.774745"
   },
   "data": {
      "Arc Melting": {
         " ": {
            "Completed By": "Daniel",
            "Finish Date": "2022-11-05",
            "Start Date": "2022-11-03",
            "Time Spent": 3
         },
         "  ": {
            "Argon Pressure": "850-900",
            "Diffusion": "Before Each Melt",
            "Ingot Location": 3,
            "Initial Purging Times": 3,
            "Vacuum Before Melt": 5e-05
         },
         "   ": {
            "Final Ingot Mass": 29.983,
            "Mass 

TypeError: cannot unpack non-iterable float object

In [ ]:
# w_aab.thin_dumps()

In [ ]:
# w_aab.thin_plots()

In [ ]:
# w_aab.dumps()

In [ ]:
# plot_graph("data/AAB")

In [ ]:
# w_aab.thin_dumps_single_obj(w_aab.terminal_materials['AAB04']['B']['VAM'])

In [ ]:
# aab_tree_paths = w_aab.make_tree(Folder, Path(w_aab.root))
# for p in aab_tree_paths:
#     print(p.displayable())

In [ ]:
# iteration_count = 1
# iteration = "AAA"
# source_folder = "../data/"
# output_folder = "data/"
# sample_data_folder = "../Sample Data/Iteration{}_{}".format(iteration_count, iteration)
# path = os.path.join(source_folder, iteration)
# w_aaa = TAMUWorkflow(path, output_folder=output_folder, iteration=iteration, sample_data_folder=sample_data_folder)

In [ ]:
# aaa_tree_paths = w_aaa.make_tree(Folder, Path(w_aaa.root))
# for p in aaa_tree_paths:
#     print(p.displayable())

In [ ]:
# aac_tree_paths = w_aac.make_tree(Folder, Path(w_aac.root))
# for p in aac_tree_paths:
#     print(p.displayable())